In [1]:
import pandas as pd 
import requests
import os
import json

In [10]:
with open(r'C:\Users\akobe\OneDrive\Documents\Keys\eliteKey.txt') as content:
    elite_key = content.read()

In [4]:
#import nhl_data
nhl_data = pd.read_csv(r'C:\Users\akobe\OneDrive\Desktop\Lighthouse\Final\Predicting_Success_First_Round_Picks_NHL\Data\nhl_data.csv', index_col = [0]) #index col makes sures another index column is not needed

In [5]:
nhl_data.head(2)

,draft_year,pick_no,team,prospect_name,prospect_id,dob,birth_country,height,weight,shoots,nhl_id,position,prospect_category,amateur_league,amateur_team,nhl_games_played,200+games
0,2009,1,New York Islanders,John Tavares,24111,1990-09-20,CAN,"6' 1""",209,L,8475166,C,North American Skater,OHL,London,1017,1
1,2009,2,Tampa Bay Lightning,Victor Hedman,33799,1990-12-18,SWE,"6' 6""",220,L,8475167,D,European Skater,SWEDEN,Modo,964,1


In [11]:
#update to the correct dob 
nhl_data.at[11, 'dob'] = '1991-05-09'

In [13]:
nhl_data['player_fname'] = nhl_data["prospect_name"].apply(lambda x: x[0:x.find(" ")])

In [14]:
nhl_data['player_lname'] = nhl_data["prospect_name"].apply(lambda x: x[x.find(" ")+1:])

In [17]:
##get this done a better way 
nhl_data.at[11, 'player_lname'] = 'De%20Haan'
nhl_data.at[195, 'player_lname'] = 'Eriksson%20Ek'
nhl_data.at[150, 'player_lname'] = 'Dal%20Colle'

In [18]:
nhl_id = nhl_data['nhl_id'].tolist()

In [217]:
draft_year = nhl_data['draft_year'].tolist()

In [50]:
nhl_data.head(2)

,draft_year,pick_no,team,prospect_name,prospect_id,dob,birth_country,height,weight,shoots,nhl_id,position,prospect_category,amateur_league,amateur_team,nhl_games_played,200+games,player_fname,player_lname
0,2009,1,New York Islanders,John Tavares,24111,1990-09-20,CAN,"6' 1""",209,L,8475166,C,North American Skater,OHL,London,1017,1,John,Tavares
1,2009,2,Tampa Bay Lightning,Victor Hedman,33799,1990-12-18,SWE,"6' 6""",220,L,8475167,D,European Skater,SWEDEN,Modo,964,1,Victor,Hedman


In [21]:
#create a list of player names - for API call 
player_fname = []
player_lname = []

player_fname = nhl_data['player_fname'].tolist()
player_lname = nhl_data['player_lname'].tolist()

In [22]:
#create a list of dob - for API call 
dob = []
dob =  nhl_data['dob'].tolist()

In [23]:
all_elite_url = []
for i in range(296):
    first_name = player_fname[i]
    last_name = player_lname[i]
    birth_date = dob[i]
    elite_url = 'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name='+first_name+'%20'+last_name+'&dateOfBirth='+str(birth_date)+'&apiKey='
    all_elite_url.append(elite_url)

In [172]:
all_elite_url[291]

'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Nicolas%20Beaudin&dateOfBirth=1999-10-09&apiKey='

In [25]:
elite_data = []

for j in range(296):
    response = requests.get(all_elite_url[j]+elite_key)
    elite_data.append(response.json())

In [252]:
from IPython.display import JSON
JSON(elite_data)

TypeError: JSON expects JSONable dict or list, not       nhl_id elite_id          elite_name   elite_dob  draft_year draft_season
0    8475166     9223        John Tavares  1990-09-20        2009    2008-2009
1    8475167     6007       Victor Hedman  1990-12-18        2009    2008-2009
2    8475168    12571        Matt Duchene  1991-01-16        2009    2008-2009
3    8475169    14657        Evander Kane  1991-08-02        2009    2008-2009
4    8475170    14739      Brayden Schenn  1991-08-22        2009    2008-2009
..       ...      ...                 ...         ...         ...          ...
291  8480814   284924     Nicolas Beaudin  1999-10-07        2018    2017-2018
292  8480878   384079      Nils Lundkvist  2000-07-27        2018    2017-2018
293  8480873   289453       Rasmus Sandin  2000-03-07        2018    2017-2018
294  8480813   271713          Joe Veleno  2000-01-13        2018    2017-2018
295  8480823   333787  Alexander Alexeyev  1999-11-15        2018    2017-2018



In [109]:
len(elite_data[0]['data'])

1

In [145]:
#creating empty lists 
elite_id = []
elite_name = []

In [146]:
for i in range(len(elite_data)):
    if len(elite_data[i]['data']) > 0:
        elite_id.append(elite_data[i]['data'][0]['id'])
        elite_name.append(elite_data[i]['data'][0]['name'])
    else:
        elite_id.append('NaN')
        elite_name.append('NaN')  

In [200]:
#create a data frame with the information 
elite_data = pd.DataFrame(list(zip(nhl_id, elite_id, elite_name, dob)),
               columns =['nhl_id', 'elite_id', 'elite_name', 'elite_dob'])

In [173]:
#fixing Nan's 
#5- Oliver Ekman Larson 
urlOEL = 'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Nicolas%20Beaudin&dateOfBirth=1999-10-07&apiKey='

response = requests.get(urlOEL+elite_key)

In [156]:
missing_data = []

In [174]:
missing_data.append(response.json())

In [210]:
#fill in missing data
elite_data.at[5, 'elite_id'] = '7693'
elite_data.at[5, 'elite_name'] = 'Oliver Ekman Larsson'
elite_data.at[5, 'elite_dob'] = '1991-07-17'
elite_data.at[29, 'elite_id'] = '30902'
elite_data.at[29, 'elite_name'] = 'Simon Després'
elite_data.at[29, 'elite_dob'] = '1991-07-27'
elite_data.at[215, 'elite_id'] = '248391'
elite_data.at[215, 'elite_name'] = 'Logan Brown'
elite_data.at[215, 'elite_dob'] = '1998-03-05'
elite_data.at[291, 'elite_id'] = '284924'
elite_data.at[291, 'elite_name'] = 'Nicolas Beaudin'
elite_data.at[291, 'elite_dob'] = '1999-10-07'

In [214]:
elite_data.head(292)

,nhl_id,elite_id,elite_name,elite_dob
0,8475166,9223,John Tavares,1990-09-20
1,8475167,6007,Victor Hedman,1990-12-18
2,8475168,12571,Matt Duchene,1991-01-16
3,8475169,14657,Evander Kane,1991-08-02
4,8475170,14739,Brayden Schenn,1991-08-22
...,...,...,...,...
287,8480806,265570,Isac Lundeström,1999-11-06
288,8480872,316794,Filip Johansson,2000-03-23
289,8480827,259973,Dominik Bokk,2000-02-03
290,8480879,281697,Jacob Bernard-Docker,2000-06-30


In [255]:
#save the data to csv 
#elite_data.to_csv('info_for_pulling_prospects.csv')

# draft year data 

In [218]:
elite_data['draft_year'] = draft_year

In [220]:
elite_data.head(2)

,nhl_id,elite_id,elite_name,elite_dob,draft_year
0,8475166,9223,John Tavares,1990-09-20,2009
1,8475167,6007,Victor Hedman,1990-12-18,2009


In [235]:
elite_data.dtypes

nhl_id         int64
elite_id      object
elite_name    object
elite_dob     object
draft_year     int64
dtype: object

In [239]:
#draft season is the year before and year of draft
#this information is needed for the API call to only get stats for their draft year 
draft_season = []

for i in elite_data['draft_year']: 
    if i == 2009:
        draft_season.append('2008-2009')
    elif i == 2010:
        draft_season.append('2009-2010')
    elif i == 2011:
        draft_season.append('2010-2011')
    elif i == 2012:
        draft_season.append('2011-2012')
    elif i == 2013:
        draft_season.append('2012-2013')
    elif i == 2014:
        draft_season.append('2013-2014')
    elif i == 2015:
        draft_season.append('2014-2015')
    elif i == 2016:
        draft_season.append('2015-2016')
    elif i == 2017:
        draft_season.append('2016-2017')
    elif i == 2018:
        draft_season.append('2017-2018')

In [241]:
elite_data['draft_season'] = draft_season

In [5]:
elite_id = []
elite_id = elite_data['elite_id'].tolist()

In [6]:
draft_season = []
draft_season = elite_data['draft_season'].tolist()

In [254]:
elite_data.head(2)

,nhl_id,elite_id,elite_name,elite_dob,draft_year,draft_season
0,8475166,9223,John Tavares,1990-09-20,2009,2008-2009
1,8475167,6007,Victor Hedman,1990-12-18,2009,2008-2009


## start here 

In [2]:
elite_data = pd.read_csv('info_for_pulling_prospects.csv', index_col = [0]) #index col makes sures another index column is not needed

In [158]:
elite_data.head()

,nhl_id,elite_id,elite_name,elite_dob,draft_year,draft_season
0,8475166,9223,John Tavares,1990-09-20,2009,2008-2009
1,8475167,6007,Victor Hedman,1990-12-18,2009,2008-2009
2,8475168,12571,Matt Duchene,1991-01-16,2009,2008-2009
3,8475169,14657,Evander Kane,1991-08-02,2009,2008-2009
4,8475170,14739,Brayden Schenn,1991-08-22,2009,2008-2009


In [7]:
#urls for draft year stats 
all_elite_prospect_url = []
for i in range(len(elite_data)):
    ident = elite_id[i]
    season = draft_season[i]
    elite_prospect_url = 'https://api.eliteprospects.com/v1/players/'+str(ident)+'/stats?offset=0&limit=100&sort=season&season='+season+'&team.teamType=club&apiKey='
    all_elite_prospect_url.append(elite_prospect_url)

In [8]:
all_elite_prospect_url

['https://api.eliteprospects.com/v1/players/9223/stats?offset=0&limit=100&sort=season&season=2008-2009&team.teamType=club&apiKey=',
 'https://api.eliteprospects.com/v1/players/6007/stats?offset=0&limit=100&sort=season&season=2008-2009&team.teamType=club&apiKey=',
 'https://api.eliteprospects.com/v1/players/12571/stats?offset=0&limit=100&sort=season&season=2008-2009&team.teamType=club&apiKey=',
 'https://api.eliteprospects.com/v1/players/14657/stats?offset=0&limit=100&sort=season&season=2008-2009&team.teamType=club&apiKey=',
 'https://api.eliteprospects.com/v1/players/14739/stats?offset=0&limit=100&sort=season&season=2008-2009&team.teamType=club&apiKey=',
 'https://api.eliteprospects.com/v1/players/7693/stats?offset=0&limit=100&sort=season&season=2008-2009&team.teamType=club&apiKey=',
 'https://api.eliteprospects.com/v1/players/14728/stats?offset=0&limit=100&sort=season&season=2008-2009&team.teamType=club&apiKey=',
 'https://api.eliteprospects.com/v1/players/13799/stats?offset=0&limit=1

In [11]:
elite_prospect_data = []

for j in range(296):
    response = requests.get(all_elite_prospect_url[j]+elite_key)
    elite_prospect_data.append(response.json())

In [12]:
from IPython.display import JSON
JSON(elite_prospect_data)

<IPython.core.display.JSON object>

In [60]:
elite_prospect_data[0]['data'][1]['regularStats']['GP']

32

In [81]:
elite_prospect_data_no_null = json.loads(json.dumps(elite_prospect_data).replace('null', '0'))

In [82]:
type(elite_prospect_data_no_null)

list

In [83]:
JSON(elite_prospect_data_no_null)

<IPython.core.display.JSON object>

In [144]:
#creating empty lists for the data 
prospect_gp = []
prospect_g = []
prospect_a = []
prospect_pts = []
prospect_pim = []
prospect_pm = []

In [145]:
# creates lists of all variables wanted, if a player was with more than one team in their draft year their stats are added together 
##this one mostly works

for prospect in elite_prospect_data_no_null[:]:
    gp = 0
    goals = 0
    assists = 0
    pts = 0
    pim = 0
    pm = 0
    
    prospect_data = prospect['data']
    i = 0
    for prospect_stats in prospect_data:
        #some players were with more than one team in their draft year, this goes through each team and adds the data together 
        i+=1
        if prospect_stats['regularStats'] == 0:   #if there is no info at regularStats, this spasses it and moves onto the next 
            pass
        else:   
            gp += prospect_stats['regularStats']['GP'] 
            goals += prospect_stats['regularStats']['G']
            assists += prospect_stats['regularStats']['A']
            pts += prospect_stats['regularStats']['PTS']
            pim += prospect_stats['regularStats']['PIM']
            pm +=  prospect_stats['regularStats']['PM']

    #appends the information to a new list         
    prospect_gp.append(gp)  
    prospect_g.append(goals)  
    prospect_a.append(assists)  
    prospect_pts.append(pts)  
    prospect_pim.append(pim)  
    prospect_pm.append(pm)  

In [151]:
len(prospect_pm)

296

In [153]:
#create a data frame with the information 
prospect_data = pd.DataFrame(list(zip(elite_id, prospect_gp, prospect_g, prospect_a, prospect_pts, prospect_pim, prospect_pm)),
               columns =['elite_id', 'prospect_gp', 'prospect_g', 'prospect_a', 'prospect_pts', 'prospect_pim', 'prospect_pm'])              

In [155]:
prospect_data.head(2)

,elite_id,prospect_gp,prospect_g,prospect_a,prospect_pts,prospect_pim,prospect_pm
0,9223,56,58,46,104,54,10
1,6007,45,7,16,23,62,21


In [157]:
#save the data to csv 
#prospect_data.to_csv('prospect_data.csv')

## Joining elite dataframe 

In [159]:
#joining tables 
all_elite_prospect_data = pd.merge(elite_data, prospect_data,  on = 'elite_id')
all_elite_prospect_data.head()

,nhl_id,elite_id,elite_name,elite_dob,draft_year,draft_season,prospect_gp,prospect_g,prospect_a,prospect_pts,prospect_pim,prospect_pm
0,8475166,9223,John Tavares,1990-09-20,2009,2008-2009,56,58,46,104,54,10
1,8475167,6007,Victor Hedman,1990-12-18,2009,2008-2009,45,7,16,23,62,21
2,8475168,12571,Matt Duchene,1991-01-16,2009,2008-2009,57,31,48,79,42,32
3,8475169,14657,Evander Kane,1991-08-02,2009,2008-2009,61,48,48,96,89,51
4,8475170,14739,Brayden Schenn,1991-08-22,2009,2008-2009,70,32,56,88,82,20


In [161]:
#save the data to csv 
#all_elite_prospect_data.to_csv('all_elite_prospect_data.csv')